<a href="https://colab.research.google.com/github/gitgitho0ray/scrappy/blob/master/10_times_scrappy_doo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from bs4 import BeautifulSoup # For HTML parsing
import requests # Website connections
from time import sleep # To prevent overwhelming the server between connections
from collections import Counter # Keep track of our term counts
import pandas as pd # For converting results to a dataframe and bar chart plots
import json # For parsing json
import re #regex
from time import strptime
import datetime
from datetime import datetime
from calendar import monthrange

In [0]:
if __name__ == '__main__':

    #do magic 
    class EventDate():

        def __init__(self, eventdate):

            sday=int(strptime(eventdate[0],'%d').tm_mday)
            month=int(strptime(eventdate[2],'%b').tm_mon)
            eday=int(strptime(eventdate[1],'%d').tm_mday)
            year=int(strptime(eventdate[-1],'%Y').tm_year)

            self.startdate = datetime.date(year, month, sday)
            self.enddate = datetime.date(year, month, eday)

    class SearchDate():   

        def __init__(self,today='',endofweek='',endofthemonth=''):
            self.today=str(datetime.now().date())
            self.addoneweek=str((datetime.now().date()+timedelta(days=7)))
            self.endofthemonth=str(datetime(datetime.now().year,datetime.now().month,monthrange(datetime.now().year, datetime.now().month)[1]).date())

    # class ScrappyDoo():


    def get_html(url, params=None):
        h = {'user-agent':'Mozilla/5.0 (Macintosh; Intel) Gecko/20100101 Firefox/74.0'}
        response = requests.get(url,params=params,headers=h)
        return response.text

    def get_content(html):
        
        soup = BeautifulSoup(get_html(url), 'html.parser')
        items = soup.find_all('tr', class_='box')
        event_links=[]
        for i in items:
            if i.find('a', {'target':'_blank'}) !=None:
                event_links.append(i.find('a', {'target':'_blank'}).get('href'))
        return event_links


    def parse():
        html = get_html(url)
        if html.status_code == 200:
            get_content(html.text)
        else:
            print('error')
        return html
    date = SearchDate()
    url = f'https://10times.com/canada/conferences?month=today&datefrom={date.today}&dateto={date.today}'

In [434]:
url

'https://10times.com/canada/conferences?month=today&datefrom=2020-05-13&dateto=2020-05-13'

In [0]:
h = {'user-agent':'Mozilla/5.0 (Macintosh; Intel …) Gecko/20100101 Firefox/74.0'}

In [436]:
get_content(get_html(url))

ConnectionError: ignored

In [382]:
eventlinksdf=pd.DataFrame(get_content(get_html(url)),columns=['URL'])
eventlinksdf.shape

(40, 1)

In [423]:

urls = eventlinksdf['URL'].to_list()
venorg=[]
for url in urls: 
    print(url)
    soup = BeautifulSoup(get_html(url), 'html.parser')
    

    
    #Event info 
    eventname = soup.find('h1').get_text().split('\n')[0]
    try:
        organizer = soup.find('h3',{'id':'org-name'}).get_text().split('\n')[0]
    except AttributeError:
        organizer = soup.find('h3').get_text().split('\n')[0]

    description = soup.find('p', class_="desc mng word-break").get_text(strip=True)
    location=[i.get_text() for i in soup.find_all('p') if i.find('span') != None][0]
    try:
        eventdate = EventDate(soup.select('span[content]')[0].get_text().replace('-','').split())
    except ValueError:
        continue

    rawtables = [i.find('table',class_='table noBorder mng').find_all('td') for i in soup.find_all('div', class_='row11')]
    info=[rawtables[0][i].get_text().split(" ",1) for i in range(len(rawtables[0]))]

    try:
        infodata=[i[1].strip('\n').strip() for i in info]
    except IndexError:
        continue

    time = [" ".join(i[:2]) for i in [i.split() for i in infodata[0].split('-')]] # time 
    participants =(re.findall(r"[0-9]+\s-\s[0-9]+ | [0-9]+", infodata[2]))[0].strip()
    tags = (', '.join(infodata[3].replace('Type','').replace('&','').split()))
    
    #ATTENDEES BLOCK 
    attendeename=[]
    profilelinks=[]
    attendeeloc=[]
    attendeedata=[]

    try:
        for i in soup.find('div', class_="visitor clearfix"):
            try:
                attendeedata.append([i.find('h4').get_text(),' '.join(i.get_text().replace('Connect','').split()[-2:]),i.find('a').get('href')])
            except AttributeError:
                continue
    except TypeError:
        continue

    #df list 
    venorg.append([eventname,organizer,description,location, eventdate.startdate,eventdate.enddate,time[0],time[1],tags,participants,attendeedata])


finaldf = pd.DataFrame(venorg,columns=['EventName','Organizer','Description','Address','StartDate','EndDate','StartTime','EndTime','Tags','ExpectedP','AttendeeData'])



https://10times.com/oaca-conference


KeyboardInterrupt: ignored

In [420]:
finaldf

,Organizer,Description,Address,StartDate,EndDate,StartTime,EndTime,Tags,ExpectedP,AttendeeData
0,Ontario Association of Committees of Adjustmen...,This conference brings together speakers who p...,5875 Falls Ave Niagara Falls ON L2G 3K7 Canada,2020-05-10,2020-05-13,09:00 AM,06:00 PM,"Conference, Education, Training",100 - 500,"[[Astrid Hudson, Toronto, Canada, https://10ti..."
1,Pandell Newly listed,Pandell Conference is the must-do users' confe...,405 Spray Ave Banff AB T1L 1J4 Canada,2020-05-10,2020-05-13,09:00 AM,06:00 PM,"Conference, IT, Technology",100 - 500,"[[barry alpha mamadou, Dakar, Senegal, https:/..."
2,Canadian Institute of Mining,The International Conference on Uranium will p...,601 Spadina Crescent E Saskatoon SK S7K 3G8 C...,2020-05-10,2020-05-13,09:00 AM,06:00 PM,"Conference, Science, Research, Power, Energy",100 - 500,"[[AKINSEHINDE TIMOTHY ADEKUNLE, Yenagoa, Niger..."
3,Strategy Institute Inc,"""Integrated Care- Transforming Patient Care an...","Toronto,\n ...",2020-05-12,2020-05-13,07:30 AM,05:00 PM,"Conference, Medical, Pharma, Wellness,, Health...",100 - 500,"[[Jamie Lai, Hill, Canada, https://10times.com..."
4,Cmhc Newly listed,National Housing Conference will connect with ...,"11 Colonel By Drive, Ottawa, ON K1N 9H4 Canada",2020-05-12,2020-05-13,09:00 AM,06:00 PM,"Conference, Building, Construction, Home, Office",100 - 500,"[[Olokede Benedict Oluseye, United Kingdom, ht..."
5,Raven Events,ABM Indigenous: West is about connecting commu...,"805 TRU Way, Kamloops, BC V2C 0C8, Canada Canada",2020-05-12,2020-05-13,09:00 AM,06:00 PM,"Conference, Travel, Tourism, Business, Services",100 - 500,"[[M G MALEK, Nadiad, India, https://10times.co..."
6,Partnership for the organization of innovation...,P4IE International Conference will bring toget...,"361 Queen St, Ottawa, ON K1R 0C7, Canada Canada",2020-05-12,2020-05-13,09:00 AM,06:00 PM,"Conference, Education, Training",100 - 500,"[[SSEMBATYA DAVID, Kampala, Uganda, https://10..."
7,INFONEX Inc.,Legal Issues in Privacy & Cyber Security will ...,"Vancouver,\n ...",2020-05-12,2020-05-13,09:00 AM,06:00 PM,"Conference, IT, Technology",100,"[[Atif Altaf Ahmed Chaudhary, Lahore, Pakistan..."
8,University Of Bayreuth,The International Meeting on Chemical Sensors ...,1001 Place Jean-Paul Riopelle Montreal QC H2X...,2020-05-10,2020-05-14,09:00 AM,06:00 PM,"Conference, Power, Energy, IT, Technology",100 - 500,"[[Gamal Turky, Cairo, Egypt, https://10times.c..."
9,Delaney Meeting & Event Management,IALE-North America Annual Meeting aims to fost...,89 Chestnut St Toronto ON M5G 2J1 Canada,2020-05-10,2020-05-14,09:00 AM,06:00 PM,"Conference, Science, Research, IT, Technology",100 - 500,"[[MUHAMMAD IFTIKHAR, Sargodha, Pakistan, https..."


In [305]:
# items = soup.find_all('div', class_='row11')
rawtables = [i.find('table',class_='table noBorder mng').find_all('td') for i in soup.find_all('div', class_='row11')]
info=[rawtables[0][i].get_text().split(" ",1) for i in range(len(rawtables[0]))]
infodata=[i[1].strip('\n').strip() for i in info]
time = [" ".join(i[:2]) for i in [i.split() for i in infodata[0].split('-')]] # time 
participants =(re.findall(r"[0-9]+\s-\s[0-9]+", infodata[2]))
tags = (', '.join(infodata[3].replace('Type','').replace('&','').split()))
print(tags)

Conference, Science, Research, Education, Training


In [296]:
userurl='https://10times.com/judy-project'
soup1 = BeautifulSoup(get_html(userurl), 'html.parser')
attendeename=[]
profilelinks=[]
attendeeloc=[]
attendeedata=[]

#ATTENDEES BLOCK 

for i in soup1.find('div', class_="visitor clearfix"):
    try:
        attendeedata.append([i.find('h4').get_text(),' '.join(i.get_text().replace('Connect','').split()[-2:]),i.find('a').get('href')])
        attendees.append(i.find('h4').get_text()) # name
        profilelinks.append(i.find('a').get('href')) #profilelink
        attendeeloc.append(' '.join(i.get_text().replace('Connect','').split()[-2:])) #location 
    except AttributeError:
        continue
attendeedata

[['Tufan YAYLA',
  'Bozüyük, Turkey',
  'https://10times.com/profile/tufan-yayla-45443537']]

In [0]:
e

In [424]:
userurl='https://10times.com/oaca-conference'
soup1 = BeautifulSoup(get_html(userurl), 'html.parser')
soup.find('h1').get_text().split('\n')

ConnectionError: ignored

2020-05-10


In [197]:
# class EventDate():

#     def __init__(self, eventdate):

#         sday=int(strptime(eventdate[0],'%d').tm_mday)
#         month=int(strptime(eventdate[2],'%b').tm_mon)
#         eday=int(strptime(eventdate[1],'%d').tm_mday)
#         year=int(strptime(eventdate[-1],'%Y').tm_year)

#         self.startdate = datetime.date(year, month, sday)
#         self.enddate = datetime.date(year, month, eday)

# eventdate = EventDate(soup.select('span[content]')[0].get_text().replace('-','').split())
# eventdate[2]=str(strptime(eventdate[2],'%b').tm_mon)
# str(strptime(eventdate[0],'%d').tm_mday)
# str(strptime(eventdate[1],'%d').tm_mday)
# # strptime('-'.join((list([str(eventdate[0])]+eventdate[2:]))),'%dd-%b-%y')


'8'